In [15]:
class Training_Data:
    
    def __init__(self, data_file):
        self.data_file = data_file
        
    def load_data(self):
        
        import os
    
        file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', self.data_file))    
        with open(file_path, 'rt', encoding='utf-8') as tf:
            loaded_ad = tf.read().split('\n')
        job_ads = []
        job_ads = [self.parse_data(item) for item in loaded_ad[:-1]]
        self.raw_data = job_ads
        self.classes = set([item['class'] for item in job_ads])
    
    def parse_data(self, input_line):
        
        cur_text = input_line.split('`')
        ret_data = {}
        ret_data['id'] = cur_text[1]
        ret_data['company'] = cur_text[3]
        ret_data['position'] = cur_text[5]
        ret_data['url'] = cur_text[7]
        ret_data['desc'] = cur_text[9]
        ret_data['class'] = ''.join(cur_text[11:]).replace(',NA', '')
    
        return ret_data
    
    def create_training_set(self, labels):
        
        def remove_white_space(in_text):
            
            on_text = in_text[:].replace(u'\xa0', ' ')
            while on_text.find('  ') > -1:
                on_text = on_text.replace('  ', ' ')
                
            return on_text
        
        self.label_dict = {}
        for i, item in enumerate(labels):
            self.label_dict[item] = i + 1
        self.sample_desc = [remove_white_space(item['desc']) for item in self.raw_data]
        self.sample_title = [remove_white_space(item['position']) for item in self.raw_data]
        self.label = [self.label_dict[item['class']] if item['class'] in self.label_dict else 0 for item in self.raw_data]
        self.label_names = [item['class'] if item['class'] in self.label_dict else 'Other' for item in self.raw_data]
        
    def balance_training_set(self, balance_class):
    
        import numpy as np
        prop = np.mean([np.array(self.label) == self.label_dict[balance_class]])
        
        if prop < 0.5:
            prop = prop / (1 - prop)
            mask = [True if item == self.label_dict[balance_class] or np.random.rand() < prop else False for item in self.label]
        else:
            prop = (1 - prop) / prop
            mask = [True if item != self.label_dict[balance_class] or np.random.rand() < prop else False for item in self.label]
        sample_desc = []
        sample_title = []
        label = []
        for i, item in enumerate(mask):
            if item:
                sample_desc.append(self.sample_desc[i])
                sample_title.append(self.sample_title[i])
                label.append(self.label[i])
        self.sample_desc = sample_desc
        self.sample_title = sample_title
        self.label = label

In [16]:
TD = Training_Data('trainingset_20180406.csv')
TD.load_data()
TD.create_training_set(TD.classes)
import pandas as pd
ad_data = pd.Series(TD.label_names)
#pd.value_counts(ad_data)

In [17]:
TD.create_training_set(['Other'])
TD.balance_training_set('Other')

In [18]:
import os
os.sys.path.append('..')
from tokenizer import Tokenizer as my_tokenizer
tkn1 = my_tokenizer(1)
tkn2 = my_tokenizer(2)
tkn3 = my_tokenizer(3)
tkn4 = my_tokenizer(4)

In [19]:
import os
with open(os.path.abspath(os.path.join(os.getcwd(), '..', 'dict', 'desc_vocab_small.txt'))  , 'rt', encoding='utf-8') as f_tv:
    desc_vocab = f_tv.read().split('\n')
with open(os.path.abspath(os.path.join(os.getcwd(), '..', 'dict', 'title_vocab_small.txt'))  , 'rt', encoding='utf-8') as f_tv:
    title_vocab = f_tv.read().split('\n')

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
desc_vectorizer = CountVectorizer(tokenizer=tkn2.tokenizer, vocabulary=desc_vocab)
desc_vec = desc_vectorizer.fit_transform(TD.sample_desc)
title_vectorizer = CountVectorizer(tokenizer=tkn4.tokenizer, vocabulary=title_vocab)
title_vec = title_vectorizer.fit_transform(TD.sample_title)

In [21]:
from scipy.sparse import hstack
print(desc_vec.shape, title_vec.shape)
data_vec = hstack([title_vec, desc_vec])
print(data_vec.shape)

(933, 19) (933, 39)
(933, 58)


In [22]:
import numpy as np
label_vec = np.array(TD.label)

In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn import metrics
clf = SVC()
scores = cross_val_score(clf, data_vec, label_vec, cv=5, scoring='accuracy')
print(scores)

[0.70053476 0.73796791 0.68449198 0.72192513 0.67027027]


In [24]:
from sklearn.model_selection import train_test_split
desc_train, desc_test, label_train, label_test = train_test_split(data_vec, label_vec, test_size=0.3)
clf = clf.fit(desc_train, label_train)
label_predict = clf.predict(desc_test)
from sklearn.metrics import classification_report
print(classification_report(label_test, label_predict, target_names=['Other', 'STEM']))

             precision    recall  f1-score   support

      Other       0.68      0.87      0.76       142
       STEM       0.81      0.57      0.67       138

avg / total       0.74      0.72      0.72       280



In [25]:
from sklearn.naive_bayes import BernoulliNB
bclf = BernoulliNB()
scores = cross_val_score(bclf, data_vec, label_vec, cv=3, scoring='f1_macro')
print(scores)
from sklearn.model_selection import train_test_split
desc_train, desc_test, label_train, label_test = train_test_split(data_vec, label_vec, test_size=0.3)
bclf = bclf.fit(desc_train, label_train)
label_predict = bclf.predict(desc_test)
from sklearn.metrics import classification_report
print(classification_report(label_test, label_predict, target_names=['Other', 'STEM']))
in_bclf = BernoulliNB()
in_bclf = in_bclf.fit(data_vec, label_vec)
label_predict = bclf.predict(data_vec)
from sklearn.metrics import classification_report
print(classification_report(label_vec, label_predict, target_names=['Other', 'STEM']))

[0.73535787 0.67979408 0.7234612 ]
             precision    recall  f1-score   support

      Other       0.62      0.90      0.73       136
       STEM       0.83      0.49      0.61       144

avg / total       0.73      0.69      0.67       280

             precision    recall  f1-score   support

      Other       0.67      0.91      0.77       464
       STEM       0.86      0.55      0.67       469

avg / total       0.77      0.73      0.72       933



In [27]:
para_bclf = in_bclf.get_params(True)
print(para_bclf)

{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}


In [26]:
prob_bclf = in_bclf.predict_proba(desc_test)
print(prob_bclf[:10])

[[0.30985848 0.69014152]
 [0.11439851 0.88560149]
 [0.569949   0.430051  ]
 [0.569949   0.430051  ]
 [0.42458562 0.57541438]
 [0.11878726 0.88121274]
 [0.62879201 0.37120799]
 [0.569949   0.430051  ]
 [0.569949   0.430051  ]
 [0.9722248  0.0277752 ]]


In [195]:
from sklearn import tree
tclf = tree.DecisionTreeClassifier()
scores = cross_val_score(tclf, data_vec, label_vec, cv=6, scoring='f1')
print(scores)
from sklearn.model_selection import train_test_split
desc_train, desc_test, label_train, label_test = train_test_split(data_vec, label_vec, test_size=0.3)
tclf = tclf.fit(desc_train, label_train)
label_predict = tclf.predict(desc_test)
from sklearn.metrics import classification_report
print(classification_report(label_test, label_predict, target_names=['other','Clerks, etc.']))
in_tclf = tclf.fit(data_vec, label_vec)
label_predict = tclf.predict(data_vec)
from sklearn.metrics import classification_report
print(classification_report(label_vec, label_predict, target_names=['other','Clerks, etc.']))

[0.6        0.63157895 0.72727273 0.66666667 0.61538462 0.81818182]
              precision    recall  f1-score   support

       other       0.67      0.75      0.71        24
Clerks, etc.       0.73      0.64      0.68        25

 avg / total       0.70      0.69      0.69        49

              precision    recall  f1-score   support

       other       0.95      1.00      0.97        86
Clerks, etc.       1.00      0.93      0.97        76

 avg / total       0.97      0.97      0.97       162



In [ ]:
import pickle

In [170]:
predict_all = bclf.predict(data_vec)
false_neg = []
for i, item in enumerate(predict_all):
    if label_vec[i,] == 0 and predict_all[i,] != 0:
        false_neg.append([TD.sample_title[i], TD.sample_desc[i]])
for item in false_neg:
    print(item[0] + '\n' + item[1] + '\n')

หัวหน้าส่วนงานบัญชี
เพศ : ชาย , หญิง อายุ(ปี) : 30 ปีขึ้นไป ระดับการศึกษา : ปริญญาตรี - ปริญญาโท ประสบการณ์(ปี) : 5ปีขึ้นไป อื่นๆ : ไม่ระบุ คุณสมบัติเพิ่มเติม เคยปิดงบการเงิน มีความรู้ทางด้านภาษี และมาตรฐานบัญชี

เจ้าหน้าที่ฝ่ายทรัพยากรมนุษย์
หน้าที่ความรับผิดชอบ -ด้านสรรหา - รับผิดชอบงานสรรหาและสวัสดิการพนักงาน งานฝึกอบรม - ดูแลงานด้านกฏระเบียบข้อบังคับ ข้อกฏหมาย - จัดทำเอกสารการฝึกอบรม - ออกบูทรับสมัครงาน - ติดต่อประสานงานหน่วยงานที่เกี่ยวข้อง - ดูแลงานธุรการสำนักงานและงาน อื่นๆ ตามมอบหมาย เพศ : ชาย , หญิง อายุ(ปี) : 21 - 25 ระดับการศึกษา : ปริญญาตรี ประสบการณ์(ปี) : 1ปีขึ้นไป อื่นๆ : ยินดีรับนักศึกษาจบใหม่ คุณสมบัติเพิ่มเติม เพศ หญิง หรือ ชาย อายุไม่เกิน 25 ปี การศึกษาระดับปริญญาตรี สาขานิติศาสตร์ มีความกระตือรือร้น มีความคิดริเริ่ม มีแนวความคิดด้านงานบุคคลสมัยใหม่ บุคลิค มนุษย์สัมพันธ์ดี มีทักษะในการสื่อสารและประสานงาน สามารถทำงานภายใต้ภาวะกดดันได้ดี มีความรู้ภาษาอังกฤษระดับ ดี สามารถใช้โปรแกรมคอมพิวเตอร์ Word, Excel, Power Point และ อื่นๆ ได้เป็นอย่างดี สามารถใช้งาน Internet ได้เป

In [95]:
for i, item in enumerate(TD.label):
    if item == 1:
        print(TD.sample_title[i] + '\n')

ธุรการ Collection - BTS หมอชิต (PP)

เจ้าหน้าที่ฝ่ายขาย ในบริษัททัวร์

Administrative Officer @iServe สาขา Central World

เจ้าหน้าที่คีย์ข้อมูล

Reservation Staff

เลขา

เจ้าหน้าที่ฝ่ายทรัพยากรมนุษย์ (HRD)

ธุรการประสานงานโครงการ จ.ชลบุรี

ธุรการประสานงานโครงการ กำแพงเพชร

Account Payable & General Ledger Officer

เจ้าหน้าที่ธุรการ (สำนักงานใหญ่)

เจ้าหน้าที่ประสานงานลูกค้าภาษาอังกฤษเคยประสานงานในคลังสินค้าหรือประสานงานกับต่างประเทศ ประสบการณ์ 1 ปีขึ้นไป ทำงานแถวBTSแบริ่ง ติดต่อ 099-969-3522 คุณเอ

พนักงานสโตร์

เจ้าหน้าที่แผนกบัญชี

เจ้าหน้าที่ฝ่ายพัฒนาบุคลากร

Staff Event

เจ้าหน้าที่แลกเปลี่ยนเงินตราต่างประเทศ : FXO (Foreign Exchange Officer) เป็นพนักงานประจำของธนาคารกสิกรไทย จำกัด(มหาชน) ประจำสนามบินสุวรรณภูมิ

เจ้าหน้าที่จัดซื้อการตลาด (ภาษาอังกฤษ)

จัดส่ง/ทำStock

เจ้าหน้าที่ฝ่ายประชาสัมพันธ์

Admin ประจำสาขา

ธุรการ ช่าง (ประจำสำนักงานวิทยุ ลาดพร้าว)

เจ้าหน้าที่บัญชี-การเงิน

เจ้าหน้าที่ฝ่ายขาย Seafreight

เจ้าหน้าที่บัญชี

ธุรการสนาม**ด่วนจำนวนมาก

Office Admin & Receptionist 

In [84]:
from sklearn.feature_selection import mutual_info_classif as mic
mutual_info_desc = mic(desc_vec, label_vec)

In [85]:
token_desc_info = []
for token in desc_vectorizer.vocabulary_:
    token_desc_info.append([token, mutual_info_desc[desc_vectorizer.vocabulary_[token]]])

In [86]:
import csv

with open('mutual_info_desc.txt', 'wt', encoding='utf-8', newline='') as of:
    csv_write = csv.writer(of, delimiter=',', dialect='excel')
    csv_write.writerows(token_desc_info)

In [ ]:
print([item for item in desc_vectorizer.vocabulary_][:10])